<a href="https://colab.research.google.com/github/michellejm/HuggingFace_RL_course/blob/main/Reinforcement_LunarLander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook for Unit 1 of Reinforcement Learning HuggingFace Course. Make a robot that can land on the moon (correctly)

In [ ]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (803 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 125044 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install pyvirtualdisplay
#virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size = (1400, 900))
virtual_display.start()

### Packages

In [ ]:
import gymnasium as gym

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

Explore gymnasium

In [ ]:
#create environment
env = gym.make("LunarLander-v2")

#reset the environment
observation, info = env.reset()

for _ in range(20):
  #random action
  action = env.action_space.sample()
  print("action taken:", action)

  #now do that action in the environment
  #from the action, get next_state, reward, terminated, truncated, and info
  observation, reward, terminated, truncated, info = env.step(action)
  print("reward:", reward)

  #if the game is terminated (i.e., we land or crash) or truncated (w a timeout)
  if terminated or truncated:
    #reset env
    print("Environment is reset")
    observation, info = env.reset()

  env.close()

action taken: 1
reward: 2.371086185972389
action taken: 1
reward: 2.243707112123276
action taken: 2
reward: -2.205778956144502
action taken: 1
reward: 2.0473591694845097
action taken: 3
reward: 1.2540593257941464
action taken: 1
reward: 1.7390889246880203
action taken: 2
reward: -2.449474424861722
action taken: 2
reward: -3.8127500923979936
action taken: 3
reward: 1.5614551832151744
action taken: 2
reward: -1.5449922900262891
action taken: 3
reward: 1.5877850910728182
action taken: 2
reward: -3.491536843453889
action taken: 0
reward: 2.4458840398473285
action taken: 0
reward: 2.3142633611286385
action taken: 0
reward: 1.1029963631094972
action taken: 2
reward: -0.5771028460873027
action taken: 2
reward: -2.7826473323659
action taken: 1
reward: 2.3618532423030856
action taken: 3
reward: 1.1200441855390022
action taken: 3
reward: -2.3685038701199503


/usr/local/lib/python3.11/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.11/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

## Summary of Lunar Lander

[the lunar lander environment documentation](https://gymnasium.farama.org/environments/box2d/lunar_lander/)... rocket tragectory

two environ versions: discrete or continuous

Landing pad: at (0,0)

Observation is a vector size 8, which is:
* horizontal pad coordinate(x)
* vertical pad coordinate(y)
* horizontal speed (x)
* vertical speed (y)
* angle
* angular speet
* boolean if left leg touched ground
* boolean if right leg touched ground


In [ ]:
#create environ w gym.make

env = gym.make("LunarLander-v2")
env.reset()

print("____OBSERVATION SPACE____")
print("Observation space shape", env.observation_space.shape)
#get a sample (random) observation
print("Sample observation", env.observation_space.sample())

____OBSERVATION SPACE____
Observation space shape (8,)
Sample observation [ 66.462875   -32.65068      3.4514165    4.141959     1.5988265
  -0.29198608   0.12012318   0.26500532]


In [ ]:
print("\n ________ACTION SPACE_________\n")
print("Action space shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())


 ________ACTION SPACE_________

Action space shape 4
Action Space Sample 0


#### 4 possible actions:
0: do nothing \\
1: fire left engine \\
2: fire main engine \\
3: fire right engine


Reward fn:
After every step, a reward is granted. Total reward of an episode is sum of all rewards for all the steps in the episode

For each step, the reward:
* is increased/decrease closer/further from landing pad
* increased/decreased slower/faster lander is moving
* decreased more lander is tilted
* increased by 10 points for each leg in contact w ground
* decreased by .03 for each frame a side engine is firing
* Decreased by .3 for each frame the main engine is firing

Addition -100 or +1-- for crashing or landing safely
An episode is a solution if it scores 200

In [ ]:
# make a bunch of different environments to have more training experiences
env = make_vec_env("LunarLander-v2", n_envs = 16)

In [ ]:
#using PPO from Stable Baseline-3
model = PPO('MlpPolicy', env, verbose=1)

#train the agent
model.learn(total_timesteps=int(2e5))

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 91       |
|    ep_rew_mean     | -185     |
| time/              |          |
|    fps             | 3688     |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 32768    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 96          |
|    ep_rew_mean          | -130        |
| time/                   |             |
|    fps                  | 1555        |
|    iterations           | 2           |
|    time_elapsed         | 42          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.009296046 |
|    clip_fraction        | 0.0922      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.00368     |
|    learnin

### Set up custom model

In [ ]:
# TODO: Define a PPO MlpPolicy architecture

model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1
)

Using cuda device


### Train custom model

In [ ]:
#train for 1,000,000 timestamps
model.learn(total_timesteps=1000000)

#save the model
model_name = 'ppo-lunar-lander-v2'
model.save(model_name)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 91.7     |
|    ep_rew_mean     | -156     |
| time/              |          |
|    fps             | 3742     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 92.3        |
|    ep_rew_mean          | -116        |
| time/                   |             |
|    fps                  | 2440        |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.008184342 |
|    clip_fraction        | 0.0668      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_variance   | -0.016      |
|    learning_rate        | 0.

### Evaluate custom model

In [ ]:
# create eval environment
eval_env = Monitor(gym.make('LunarLander-v2', render_mode='rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward = {mean_reward:.2f} +/- {std_reward}")

mean_reward = 267.39 +/- 36.182161116135894


In [ ]:
notebook_login()
!git config --global credential.helper store

In [ ]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

repo_id = 'mmcsweeney/ppo-LunarLander-v2'

env_id = 'LunarLander-v2'

eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode = 'rgb_array'))])

model_architecture = "PPO"

commit_meddage = "Reinforcement Learning Course"

package_to_hub(model=model,
               model_name=model_name,
               model_architecture=model_architecture,
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message=commit_meddage)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpxbi1pkwq/-step-0-to-step-1000.mp4


/usr/local/lib/python3.11/dist-packages/moviepy/config_defaults.py:1: DeprecationWarning: invalid escape sequence '\P'
  """


Moviepy - Building video /tmp/tmpxbi1pkwq/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpxbi1pkwq/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpxbi1pkwq/-step-0-to-step-1000.mp4
ℹ Pushing repo mmcsweeney/ppo-LunarLander-v2 to the Hugging Face
Hub


policy.optimizer.pth:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

...

ppo-lunar-lander-v2.zip:   0%|          | 0.00/148k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

replay.mp4:   0%|          | 0.00/172k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/mmcsweeney/ppo-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/mmcsweeney/ppo-LunarLander-v2/commit/438f6e4faf4bdba268eef5f14ce4949878e1b7e4', commit_message='Reinforcement Learning Course', commit_description='', oid='438f6e4faf4bdba268eef5f14ce4949878e1b7e4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mmcsweeney/ppo-LunarLander-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='mmcsweeney/ppo-LunarLander-v2'), pr_revision=None, pr_num=None)